<a href="https://colab.research.google.com/github/MiguelAngeloTr/BIGDATA/blob/main/C1/Parcial1/Parcial_1_BIGDATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


##Universidad Autónoma de Occidente

##Facultad de Ingenierías y Ciencias Básicas

##Curso: Big Data

##Autor: Sergio A. Cantillo

##Correo: sacantillo@uao.edu.co

---

### **NOMBRE COMPLETO:Miguel Angel Jimenez Trochez**

###**CODIGO:2215407**

### **PROGRAMA:Ingenieria Informatica**

---

### **Condiciones**

*   Pueden usar los archivos Google Colab trabajados en clase.
*   No pueden usar IA para buscar las respuetas del examen. Quien sea visto en flagrancia, el examen será **anulado**.
*   Se contestan las preguntas 1, 2 y 3 Obligatoriamente, del resto se escogen 2 para hacer 5. Si contestan todas las 6 preguntas correctamente bonifican para el segundo parcial.
*   Pueden usar el material de referencia de UAO virtual
*   Subir el archivo con extensión .ipynb a la plataforma antes de las 12:59 PM.


# **Examen 1**

El conjunto de datos AB_NYC_2019.csv contiene información acerca de los listados de Airbnb en la ciudad de Nueva York. Incluye variables como el nombre del alojamiento, el nombre del anfitrión, la ubicación geográfica (vecindario y coordenadas), el tipo de habitación, el precio por noche, el número mínimo de noches requeridas para una reserva, la disponibilidad durante el año, entre otros detalles relevantes. El objetivo en este análisis de datos consiste en utilizar las herramientas computacionales y estadísticas vistas hasta el momento en el curso que logren contribuir a la caracterización de estos alojamientos



In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.5.5/spark-3.5.5-bin-hadoop3.tgz
!tar xf spark-3.5.5-bin-hadoop3.tgz
!pip install pyspark
!pip install py4j
!pip install findspark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,369 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,668 kB]
Get:13 http://archive.ubuntu.com/ubuntu jammy-up

In [2]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.5-bin-hadoop3"

In [3]:
import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Parcial 1 Spark") \
       .getOrCreate()

Comprobar sesión

In [4]:
spark

In [5]:
from pyspark.sql.types import *
from pyspark.sql.functions import to_date
from pyspark.sql.functions import countDistinct
from pyspark.sql.functions import col, regexp_extract, avg, split

In [20]:
#cargar dataset

import requests
path = "https://raw.githubusercontent.com/MiguelAngeloTr/BIGDATA/refs/heads/main/C1/Parcial1/AB_NYC_2019.csv"
req = requests.get(path)
url_content = req.content

csv_file_name = 'AB_NYC_2019.csv'
csv_file = open(csv_file_name, 'wb')

csv_file.write(url_content)
csv_file.close()





+----+--------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|       host_name|neighbourhood_group|     neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|            John|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  149|             1|              9.0| 2018-10-19|             0.21|                             6|           365.0|
|2595|Skylit Midtown Ca.

In [22]:
schema = StructType([
    StructField("id", IntegerType(), True),
    StructField("name", StringType(), True),
    StructField("host_id", IntegerType(), True),
    StructField("host_name", StringType(), True),
    StructField("neighbourhood_group", StringType(), True),
    StructField("neighbourhood", StringType(), True),
    StructField("latitude", DoubleType(), True),
    StructField("longitude", DoubleType(), True),
    StructField("room_type", StringType(), True),
    StructField("price", IntegerType(), True),
    StructField("minimum_nights", IntegerType(), True),
    StructField("number_of_reviews", IntegerType(), True),
    StructField("last_review", StringType(), True),
    StructField("reviews_per_month", DoubleType(), True),
    StructField("calculated_host_listings_count", IntegerType(), True),
    StructField("availability_365", IntegerType(), True)
])

In [23]:
df = spark.read.csv('/content/'+csv_file_name, header=True, inferSchema=True, multiLine=True,schema=schema)
df.show()

+----+--------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|  id|                name|host_id|       host_name|neighbourhood_group|     neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|last_review|reviews_per_month|calculated_host_listings_count|availability_365|
+----+--------------------+-------+----------------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------+-----------------+------------------------------+----------------+
|2539|Clean & quiet apt...|   2787|            John|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  149|             1|                9| 2018-10-19|             0.21|                             6|             365|
|2595|Skylit Midtown Ca.

In [24]:
df.printSchema()

root
 |-- id: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- host_id: integer (nullable = true)
 |-- host_name: string (nullable = true)
 |-- neighbourhood_group: string (nullable = true)
 |-- neighbourhood: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- room_type: string (nullable = true)
 |-- price: integer (nullable = true)
 |-- minimum_nights: integer (nullable = true)
 |-- number_of_reviews: integer (nullable = true)
 |-- last_review: string (nullable = true)
 |-- reviews_per_month: double (nullable = true)
 |-- calculated_host_listings_count: integer (nullable = true)
 |-- availability_365: integer (nullable = true)



Manejo de faltantes

In [14]:
def faltantes(df):
  falta = df.select([F.sum(F.col(c).isNull().cast('int')).alias(c) for c in df.columns])
  return falta.show()

## **1.**   A partir de la carga del conjunto de datos crudo, realice los procesos de adecuación (Ej. tipos de dato) y limpieza de (Ej. identificación y eliminación de columnas) requeridos.

##Responda:

## 1.1 ¿Cual es el tamaño (filas,columnas) del conjunto de datos crudo?, y luego de los procesos solicitados?

## 1.2 Indique brevemente si fueron eliminadas columnas de la base de datos y el criterio correspondiente.

---

**1.1: Respuesta**

In [25]:
print('Tamaño (filas, columnas) del conjunto de datos crudo: ', (df.count(), len(df.columns)))

Tamaño (filas, columnas) del conjunto de datos crudo:  (48895, 16)


1.2 Eliminación de columnas y filas con datos faltantes

In [27]:
df2= df.drop('id','host_id','last_review').show()
print('Tamaño (filas, columnas) del conjunto de datos con columnas eliminadas: ', (df2.count(), len(df.columns)))

+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
|  id|host_id|neighbourhood_group|     neighbourhood|latitude|longitude|      room_type|price|minimum_nights|number_of_reviews|reviews_per_month|calculated_host_listings_count|availability_365|
+----+-------+-------------------+------------------+--------+---------+---------------+-----+--------------+-----------------+-----------------+------------------------------+----------------+
|2539|   2787|           Brooklyn|        Kensington|40.64749|-73.97237|   Private room|  149|             1|                9|             0.21|                             6|             365|
|2595|   2845|          Manhattan|           Midtown|40.75362|-73.98377|Entire home/apt|  225|             1|               45|             0.38|                             2|             355|
|3647|   4632|          Manhat

## **2.** A partir de la variable precio del alojamiento (price), obtenga los coeficientes de asimetria y forma, ademas grafíquela a través de un histograma y un diagrama de cajas.

## Responda:

## 2.1 De acuerdo a los coeficientes de asimetria y forma, ¿Qué clasificación presenta esta variable? ¿Corresponde a una distribución normal convencional?

---



## **3.**   Realice una tabla donde se determine la distribución de los tipos de alojamiento disponibles (room_type) por vecindario (neighborhood_group), incluyendo el precio promedio para cada uno. Asimismo, Grafique esta distribución con un diagrama de barras donde el eje X sean los vecindarios, el eje Y las cantidades. Ademas que existan barras por cada tipo de alojamiento.

## Responda:

## 3.1 ¿Qué se puede decir acerca de la relación entre el precio y tipo de alojamiento respecto del vecindario donde estan ubicados?

## 3.2 ¿Qué vecindarios presentan la mayor cantidad de alojamientos tipo Private room?

## 3.3 ¿Qué se puede decir de la distribución de los alojamientos en manhattan con respecto a los demas vecindarios?

---



## **4.**   Realice una tabla donde se determine por vecindario (neighbourhood_group) la cantidad de alojamientos totales, su equivalente porcentual y su disponibilidad promedio (availability_365). Ordene los resultados de acuerdo al porcentaje (mayor a menor)

## Responda:

## 4.1. Teniendo en cuenta la información de esta tabla, ¿En cuál vecindario seria más probable encontrar alojamiento disponible?

## 4.2 ¿Qué se puede decir acerca de la relación de los alojamientos totales y la disponibilidad promedio en los vecindarios?

---




## **5.** Construya una matriz de correlación considerando el precio, la disponibilidad, y el numero de noches minimo de alojamiento.
## Responda:
## 5.1 ¿Cómo afecta la disponibilidad (minimum_nights) al precio de los alojamientos en Airbnb NYC?

## **6.** A partir de la variable availability_365, realice una recodificación las siguientes condiciones y añadala a dataframe con el nombre de "availability level":

* Low disp: 0 a 119 días
* Med disp: 120 a 240 días
* High disp: 241 a 360 días

## 6.1 A partir de este nuevo dataframe, entregue una tabla donde por este nivel de disponibilidad se muestre el total de alojamientos. ¿Que se puede decir acerca de esta información?